compare calibration result and simulation result in the instant loop detector

In [65]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-01'
detector = 'w2e_out'
number = 0
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [66]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [67]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,w2e_out,1.577837e+09,enter,0_w2e_out,7.02,5.00,DEFAULT_VEHTYPE,None
2,w2e_out,1.577837e+09,enter,1_w2e_out,9.86,5.00,DEFAULT_VEHTYPE,183.11
4,w2e_out,1.577837e+09,enter,2_w2e_out,12.01,5.00,DEFAULT_VEHTYPE,1.82
6,w2e_out,1.577837e+09,enter,3_w2e_out,8.40,5.00,DEFAULT_VEHTYPE,130.45
8,w2e_out,1.577837e+09,enter,4_w2e_out,11.56,5.00,DEFAULT_VEHTYPE,137.65
...,...,...,...,...,...,...,...,...
3401,w2e_out,1.577923e+09,enter,1353_w2e_out,12.76,5.00,DEFAULT_VEHTYPE,55.68
3403,w2e_out,1.577923e+09,enter,1354_w2e_out,12.80,5.00,DEFAULT_VEHTYPE,1.20
3406,w2e_out,1.577923e+09,enter,1355_w2e_out,6.33,5.00,DEFAULT_VEHTYPE,38.09
3408,w2e_out,1.577923e+09,enter,1356_w2e_out,11.03,5.00,DEFAULT_VEHTYPE,145.16


In [68]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_w2e_out,1.577837e+09,7.019594,0.842688,1577836839,1577836778,max,5.833333,0.25,1.186260
1,1_w2e_out,1.577837e+09,9.861873,1.183898,1577837023,1577836979,max,10.000000,0.07,-0.138127
2,2_w2e_out,1.577837e+09,10.453261,3.027113,1577837025,1577837003,max,11.388889,1.43,-0.935628
3,3_w2e_out,1.577837e+09,8.401513,1.008585,1577837157,1577837105,max,7.222222,-0.74,1.179291
4,4_w2e_out,1.577837e+09,11.555531,1.387219,1577837295,1577837257,max,10.833333,-0.49,0.722197


In [69]:
print(len(data), len(df_instant_out))

1358 1358


compare time error and speed error in loop detector between simulation result and calibration result

In [70]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_w2e_out,1.577837e+09,7.02,1.577837e+09,7.019594,1577836839,5.833333
1,1_w2e_out,1.577837e+09,9.86,1.577837e+09,9.861873,1577837023,10.000000
2,2_w2e_out,1.577837e+09,12.01,1.577837e+09,10.453261,1577837025,11.388889
3,3_w2e_out,1.577837e+09,8.40,1.577837e+09,8.401513,1577837157,7.222222
4,4_w2e_out,1.577837e+09,11.56,1.577837e+09,11.555531,1577837295,10.833333


In [71]:
#compare_df = compare_df.head(19)

In [72]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(1.116899852724595)

In [73]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.8715979366485079)

In [74]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(1.3585949240674728)

In [75]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(1.5862092947144493)

In [76]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}_new.html")

In [77]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-21.299999952316284 7.140000104904175
-12.549999952316284 8.97000002861023


In [78]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-6.944444444444445 12.355555555555556
-6.9749498937674295 9.862625038673595


In [79]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}_new.html")

In [80]:
#import os
#os.environ["HORNSGATAN_HOME"]
